In [1]:
%pylab inline
from pandas import Series, DataFrame
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》
作者: Wes McKinney
译者: 唐学韬

层次化索引

层次化索引（hierarchical index）是pandas的重要功能，

通过在一个轴上拥有两个以上的索引级别， 使得 Pandas 可以用低维度形式处理高维度。

层次化索引在数据重塑和基于分组操作（如透视表生成）中扮演者重要的角色。
 
# 根据索引选择数据子集

In [2]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1   -0.291810
   2   -2.489879
   3   -0.993844
b  1    0.791313
   2   -1.233354
   3    1.096154
c  1    1.627453
   2    0.637391
d  2   -0.726163
   3   -2.117156
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

In [10]:
# 以外层索引的方式选择
data['b']

1    0.791313
2   -1.233354
3    1.096154
dtype: float64

In [5]:
data['b':'c']

b  1    0.791313
   2   -1.233354
   3    1.096154
c  1    1.627453
   2    0.637391
dtype: float64

In [6]:
data.ix[['b', 'd']]

b  1    0.791313
   2   -1.233354
   3    1.096154
d  2   -0.726163
   3   -2.117156
dtype: float64

In [11]:
# 以内层索引的方式选择
data[:, 2]

a   -2.489879
b   -1.233354
c    0.637391
d   -0.726163
dtype: float64

# `stack()` 和 `unstack()`

In [13]:
# 层次化索引数据可以转换为一个DataFrame
data.unstack()

,1,2,3
a,-0.291810,-2.489879,-0.993844
b,0.791313,-1.233354,1.096154
c,1.627453,0.637391,NaN
d,NaN,-0.726163,-2.117156


In [14]:
#  反向
data.unstack().stack()

a  1   -0.291810
   2   -2.489879
   3   -0.993844
b  1    0.791313
   2   -1.233354
   3    1.096154
c  1    1.627453
   2    0.637391
d  2   -0.726163
   3   -2.117156
dtype: float64

# DataFrame 的层次化索引

In [19]:
# 对于DataFrame，横轴和竖轴都可以有层次化索引

frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [20]:
#  每层索引都可以命名
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [22]:
# 可以单独创建MultiIndex然后复用
columns = pd.MultiIndex.from_arrays([['Ohio','Ohio','Colorado'],['Green','Red','Green']],names = ['state','color'])
frame1 = DataFrame(np.arange(12).reshape((4,3)),columns = columns)
frame1

state  Ohio     Colorado
color Green Red    Green
0         0   1        2
1         3   4        5
2         6   7        8
3         9  10       11

# 重排分级顺序 

In [26]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [24]:
frame.sortlevel(1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [25]:
frame.swaplevel(0, 1).sortlevel(0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

# 根据级别汇总统计 

In [27]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#  使用 DataFrame 列

In [29]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [31]:
# set_index(): 将列转化为索引
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
# 转化为索引，同时保留列
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [35]:
# reset_index(): 将索引转化为列
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
